In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import os
# ensure folder exists
os.makedirs("/content/agri_project", exist_ok=True)

# Show what was uploaded in /content
print("Files in /content:")
!ls -la /content | sed -n '1,120p'


In [ ]:
import zipfile, os

zip_path = "/content/Project5_Ag_Crop and weed detection.zip"  # <- replace if your uploaded name is different
extract_path = "/content/agri_project"
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_path)
    print("✅ Extracted to", extract_path)
else:
    print("❗ ZIP not found at", zip_path)


In [ ]:
import os
project_path = "/content/agri_project"
print("All files in project:")
for root, dirs, files in os.walk(project_path):
    for f in files:
        print(os.path.join(root, f))


In [ ]:
# Inspect a file quickly. Replace with any .py or .txt from the list you saw.
file_to_view = "/content/agri_project/README.txt"  # <- REPLACE with a real file from Cell 4
if os.path.exists(file_to_view):
    !sed -n '1,200p' "{file_to_view}"
else:
    print("File not found:", file_to_view)


In [ ]:
import os

# Dataset paths
base_path = "/content/agri_project/project_files/agri_data/data"
classes_path = "/content/agri_project/project_files/classes.txt"

print("✅ Dataset path set.")
print("Images + labels inside:", base_path)


In [ ]:
# Check what classes exist
with open(classes_path, "r") as f:
    classes = [line.strip() for line in f.readlines() if line.strip()]
print("Classes:", classes)


In [ ]:
import glob

# List of all image files
image_files = sorted(glob.glob(os.path.join(base_path, "*.jpeg")))
label_files = sorted(glob.glob(os.path.join(base_path, "*.txt")))

print(f"Found {len(image_files)} images and {len(label_files)} labels")

# Preview a few
print("Sample image:", image_files[:3])
print("Sample label:", label_files[:3])


In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

X, y = [], []

for img_path in image_files:
    label_path = img_path.replace(".jpeg", ".txt")
    if not os.path.exists(label_path):
        continue

    # Read image
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128, 128))  # resize to fixed shape
    img = img / 255.0                  # normalize
    X.append(img)

    # Read label (first number in file)
    with open(label_path, "r") as f:
        label_line = f.readline().strip()
        if label_line: # Ensure the line is not empty
            class_id = int(label_line.split(" ")[0]) # Extract the first number as class ID
            y.append(class_id)

X = np.array(X)
y = np.array(y)

print("✅ Data prepared:", X.shape, y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Train size:", len(X_train), "Test size:", len(X_test))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # binary classification (crop vs weed)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32
)


In [ ]:
import matplotlib.pyplot as plt

loss, acc = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {acc:.2f}")

plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Model Accuracy')
plt.legend()
plt.show()


In [ ]:
model.save("/content/agri_project/crop_weed_model.h5")
print("✅ Model saved at /content/agri_project/crop_weed_model.h5")


In [ ]:
from google.colab import files
files.download("/content/agri_project/crop_weed_model.h5")
